In [2]:
import pandas as pd
from pyntcloud import PyntCloud
import numpy as np
import pdal
import shapely.wkt

In [11]:
def get_points(ept_path, bounds, wkt):
    
    READ_PIPELINE = """
                    {{
                        "pipeline": [
                            {{
                                "type": "readers.ept",
                                "filename": "{path}",
                                "bounds": "{bounds}"
                            }},
                            {{
                                "type":"filters.crop",
                                "polygon":"{wkt}"
                            }}
                        ]
                    }}
                    """
    
    pipeline = pdal.Pipeline(READ_PIPELINE.format(
        path=ept_path,
        bounds=bounds,
        wkt=wkt)
    )

    pipeline.validate()
    pipeline.execute()
    point_cloud = pipeline.arrays[0]

    return point_cloud



def write_to_laz(point_cloud, path):
    
    WRITE_PIPELINE = """
    {{
        "pipeline": [
            {{
                "type": "writers.las",
                "filename": "{path}",
                "extra_dims": "all"
            }}
        ]
    }}
    """
    pipeline = pdal.Pipeline(
        WRITE_PIPELINE.format(path=path),
        arrays=[point_cloud]
    )
    pipeline.validate()
    pipeline.execute()


In [5]:
test_area = [117813.6,487266.1,117942.1,487370.7]
xmin,ymin,xmax,ymax = test_area

path_a10_2018 = '/var/data/rws/data/2018/entwined/ept.json'
path_a10_2019 = '/var/data/rws/data/2019/amsterdam_entwined/ept.json'

# wkt = 'POLYGON((117813.6 487266.1,117813.6 487370.7,117942.1 487370.7,117942.1 487266.1,117813.6 487266.1))'
# wkt_small = 'POLYGON((117813 487266,117813 487270,117815 487270,117815 487266,117813 487266))'
wkt_poral = 'POLYGON((118032.6 488782.8,118032.6 488793.5,118078.8 488793.5,118078.8 488782.8,118032.6 488782.8))'
xmin,ymin,xmax,ymax = shapely.wkt.loads(wkt_poral).bounds

bounds = f'([{xmin}, {xmax}], [{ymin}, {ymax}])'
pc1 = get_points(path_a10_2018, bounds, wkt_poral)
print(f'loaded pc 1')

loaded pc 1


In [ ]:
pcdf = pd.DataFrame(np.array(pc1[['X','Y','Z','Red','Green','Blue']].tolist()), 
            columns=['x', 'y', 'z','red','green','blue'])



In [ ]:
pcdf = pd.DataFrame(np.array(pc1[['X','Y','Z']].tolist()), 
            columns=['x', 'y', 'z'])



In [ ]:
py = PyntCloud(pcdf)
# py.plot(mesh=True, backend="threejs")

py

In [ ]:
# py_cloud = py.get_sample("mesh_random", 
#                          n=100000, 
#                          rgb=True)#, 
#                          #as_PyntCloud=True)
    
# scene = py_cloud.plot(return_scene=True)

In [6]:
import numpy as np
import pandas as pd
import matplotlib
pd.options.mode.chained_assignment = None  # default='warn'
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from pyntcloud import PyntCloud
from sklearn.cluster import DBSCAN


min_size = 3
ransac_distance = 0.2
ransac_iterations = 1000
dbscan_distance = 1
dbscan_min_samples = 8


points = pd.DataFrame(
    np.array(pc1[['X','Y','Z']].tolist()), 
    columns=['x', 'y', 'z']
)

# points = np.array((x, y, z)).T
# points=pd.DataFrame(points,columns=['x','y','z'])
roof = PyntCloud(points)
#plpy.warning('Roofpoints in: ',len(roof.points))

eps1 = 1.2
min_size1 = 8
eps2 = 0.8
min_size2 = 6

In [7]:
def find_clusters(points, eps, min_samples):
    df = points[["x", "y","z"]]
    db = DBSCAN(eps=eps, min_samples=min_samples)
    dbsc = db.fit(df)
    labels = dbsc.labels_
    core_samples = np.zeros_like(labels, dtype = bool)
    core_samples[dbsc.core_sample_indices_] = True
    points["coresample"]= core_samples
    points["cluster"] = dbsc.labels_
    return points

def find_plane(points):
    pointcloud = PyntCloud(points)
    pointcloud.add_scalar_field("plane_fit",max_dist=ransac_distance, max_iterations=ransac_iterations, n_inliers_to_stop=None)
    inliers = pointcloud.points[pointcloud.points.is_plane == 1]
    outliers = pointcloud.points[pointcloud.points.is_plane == 0]
    return inliers, outliers

global planes
planes=pd.DataFrame(columns=['x','y','z','cid'])
def recurse(roof,cid):
    global planes
    #plpy.warning('Iter ',cid,' with ',len(roof.points),'points')
    rest=pd.DataFrame(columns=['x','y','z'])
    
    points = find_clusters(roof.points,eps1,min_size1) #split clusters and remove garbage
    clusters = points[(points.cluster >= 0)]
    _ = points[(points.cluster < 0)]
    counts = np.bincount(clusters['cluster'])
    #max = np.argmax(counts)
    #maxcluster = clusters[clusters.cluster == max]
    #restcluster = clusters[clusters.cluster != max]
    
    if len(clusters) > 0:
      planeinliers, planeoutliers = find_plane(clusters) #find best plane, keep remainder
      
      points = find_clusters(planeinliers,eps2,min_size2) #split plane in possible clusters, keep remainder
      clusterinliers = points[(points.cluster >= 0) & (points.coresample == True)]
      clusteroutliers = points[(points.cluster < 0) | ((points.cluster < 0) & (points.coresample == False))]

      rest = pd.concat([rest,planeoutliers,clusteroutliers]) #keep all remainders for further use
      if len(clusterinliers) > min_size:
          #print(' ',clusterinliers['cluster'].max() + 1, 'clusters will be added')
          for name, group in clusterinliers.groupby('cluster'): 
              cid = cid + 1
              group['cid'] = cid #assign a value to the newly found cluster
              planes = pd.concat([planes,group]) #add to the plane stack
    else:
      return
    #plpy.warning('Number of rest points: ', len(rest))
    if len(rest) <= min_size or cid > 100 or len(rest) == len(roof.points):
      return
    else:
      recurse(PyntCloud(rest),cid)

In [8]:
roof.points["cid"] = -1
recurse(roof,0)
points = np.array((planes['x'], planes['y'],planes['z'],planes['cid'])).T



In [25]:
print(np.unique(planes['cid'], return_counts =True))
c1 = points[points[:,3] == 1][:,0:3]
c2 = points[points[:,3] == 2][:,0:3]


(array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
       71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
       88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102],
      dtype=object), array([231183, 231114,  61879,  40769,   1723,      7,   1088,    634,
          448,     15,    390,    684,    149,    829,    515,     22,
           16,      4,   9434,   8011,   2508,  11133,   7600,    428,
          974,   3710,     13,   3899,   9078,   1497,   8154,    611,
          400,   1362,    209,    269,  12607,   5298,      3,     39,
           28,     42,     27,     23,     18,     13,     13,     22,
           28,      7,      6,    142,    140,     61,      8,     20,
           50,     14,    116,   824

In [33]:
print(c1.shape)
write_to_laz(c1, '/var/data/rws/data/output/tst_portal_c1.laz')
print(c2.shape)
write_to_laz(c2, '/var/data/rws/data/output/tst_portal_c2.laz')
print('done')

(3, 231183)


RuntimeError: Array without fields must have 3 dimensions.

In [26]:
pyntcloud.scalar_fields.PlaneFit(*, pyntcloud, max_dist=0.0001,
max_iterations=100, n_inliers_to_stop=None)

SyntaxError: invalid syntax (<ipython-input-26-89eca175caa0>, line 1)

In [27]:
print('hi')

hi
